# Notebook 06 – Graph RAG (RAG con Knowledge Graph)

Objetivo:
Construir un sistema que:
- Extraiga entidades y relaciones
- Construya un grafo de conocimiento
- Permita razonamiento estructural con LLM


"""
Entidad:
- name
- type (concept, method, author, field)

Relación:
- source
- target
- relation (uses, improves, based_on)
"""


📁backend/app/models/graph/knowledge_graph.py

In [ ]:
import networkx as nx

class KnowledgeGraph:
    def __init__(self):
        self.graph = nx.DiGraph()

    def add_entity(self, name: str, entity_type: str):
        self.graph.add_node(name, type=entity_type)

    def add_relation(self, source: str, target: str, relation: str):
        self.graph.add_edge(source, target, relation=relation)

    def get_subgraph(self, nodes: list[str]):
        return self.graph.subgraph(nodes)

    def to_text(self) -> str:
        lines = []
        for u, v, data in self.graph.edges(data=True):
            lines.append(f"{u} --[{data['relation']}]--> {v}")
        return "\n".join(lines)


📁backend/app/models/graph/graph_rag.py

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from app.models.llms.groq_llm import get_groq_llm
from app.models.graph.knowledge_graph import KnowledgeGraph
import json

llm = get_groq_llm()

extract_prompt = ChatPromptTemplate.from_template("""
Extrae entidades y relaciones del siguiente texto científico.

Devuelve SOLO JSON con este formato:
{
  "entities": [{"name": "...", "type": "..."}],
  "relations": [{"source": "...", "target": "...", "relation": "..."}]
}

Texto:
{input}
""")

def build_graph_from_text(text: str) -> KnowledgeGraph:
    response = llm.invoke(extract_prompt.format(input=text))
    data = json.loads(response.content)

    kg = KnowledgeGraph()

    for ent in data["entities"]:
        kg.add_entity(ent["name"], ent["type"])

    for rel in data["relations"]:
        kg.add_relation(rel["source"], rel["target"], rel["relation"])

    return kg


📁backend/app/models/graph/graph_rag.py

In [ ]:
query_prompt = ChatPromptTemplate.from_template("""
Usa el siguiente grafo de conocimiento para responder la pregunta.

Grafo:
{graph}

Pregunta:
{question}
""")

def query_graph(kg: KnowledgeGraph, question: str) -> str:
    graph_text = kg.to_text()
    return llm.invoke(
        query_prompt.format(graph=graph_text, question=question)
    ).content


EJEMPLO (SOLO NOTEBOOK)

In [ ]:
text = """
CRISPR es una tecnología basada en mecanismos bacterianos
que permite la edición genética y se usa en terapias génicas.
"""

kg = build_graph_from_text(text)

print(query_graph(kg, "¿Para qué se usa CRISPR?"))

📁backend/app/services/rag_service.py

In [ ]:
from app.models.graph.graph_rag import build_graph_from_text, query_graph

def graph_rag_answer(text: str, question: str) -> str:
    kg = build_graph_from_text(text)
    return query_graph(kg, question)